### 바이앤홀드 (buyandhold)
- 매수 후 일정 기간 동안 유지한 뒤 매도

In [ ]:
import pandas as pd

In [ ]:
# csv 폴더 안에 있는 AAPL.csv 로드 
df = pd.read_csv("../../csv/AAPL.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# 데이터프레임 정보를 확인 
df.info()

In [ ]:
# 결측치의 개수를 확인 
df.isna().sum()

In [ ]:
# 결측치가 존재하는 데이터를 확인 
# 인덱스를 기준으로 결측치가 존재하는 데이터를 확인 
# 인덱스의 조건식으로 결측치 존재(isna()) 유무(any(axis=1))
flag = df.isna().any(axis=1)

In [ ]:
df.loc[flag, ]

In [ ]:
# 결측치를 제외한 데이터프레임을 출력 
df.loc[~flag, ]

In [ ]:
df.dropna()

In [ ]:
import numpy as np

In [ ]:
# isin() : 포함 여부 
# 결측치(np.nan), 양의 무한대(np.inf), 음의 무한대(-np.inf) 포함되어있는가?
flag2 = df.isin( [np.nan, np.inf, -np.inf] ).any(axis=1)

In [ ]:
df = df.loc[~flag2, ]

In [ ]:
# Date컬럼을 시계열 데이터로 변경
df['Date'] = pd.to_datetime( df['Date'] )
# Date 컬럼을 인덱스로 변경
df.set_index('Date', inplace=True)

In [ ]:
df.head()

In [ ]:
# df에서 Adj Close 컬럼을 제외한 나머지 컬럼은 삭제 
AAPL = df[['Adj Close']]
AAPL

In [ ]:
# 일별 수익율이라는 컬럼을 생성 
# 오늘의 수정종가 / 전날의 수정종가 -> 일별 수익율
AAPL['daily_rtn'] = \
    (AAPL['Adj Close'] / AAPL['Adj Close'].shift()).fillna(1)

In [ ]:
AAPL.head()

In [ ]:
# 수익율 계산
# 일별 수익율을 모두 누적곱

rtn = 1

for i in range(len(AAPL)):
    # i가 의미하는것은? 데이터프레임의 위치
    rtn *= AAPL.iloc[i, 1]

rtn

In [ ]:
AAPL.tail()

In [ ]:
AAPL.iloc[-1, 0] / AAPL.iloc[0, 0]

In [ ]:
# pct_change() 함수를 이용하여 일별 수익율 
AAPL['daily_rtn2'] = \
    (AAPL['Adj Close'].pct_change() + 1).fillna(1)

In [ ]:
# cumprod() : 누적곱 함수 
AAPL['daily_rtn2'].cumprod()

In [ ]:
from datetime import datetime

In [ ]:
# 인덱스를 기준으로 데이터를 필터링 후 수익율 계산
# 시작시간 생성 
start_time = "2010-01-01"
end_time = "2015-12-31"
# start_time과 end_time를 시계열 데이터로 변경 
start = pd.to_datetime(start_time)
end = datetime.strptime(end_time, '%Y-%m-%d')

In [ ]:
price_AAPL = AAPL.loc[start : end, ['Adj Close']]

In [ ]:
# 일별 수익율 컬럼을 생성
price_AAPL['daily_rtn'] = \
    (price_AAPL['Adj Close'] / price_AAPL['Adj Close'].shift()).fillna(1)

In [ ]:
# pct_change() 함수를 이용하여 일별 수익율 생성 
price_AAPL['daily_rtn2'] = (price_AAPL['Adj Close'].pct_change() + 1).fillna(1)

In [ ]:
price_AAPL.head()

In [ ]:
# 누적 수익율 계산
rtn = 1

for i in range(len(price_AAPL)):
    rtn *= price_AAPL.iloc[i, 1]

rtn

In [ ]:
price_AAPL['rtn'] = price_AAPL['daily_rtn2'].cumprod()

In [ ]:
price_AAPL.tail(1)

In [ ]:
price_AAPL.iloc[-1, 0] / price_AAPL.iloc[0, 0]

In [ ]:
# BuyAndHold 함수를 생성 
# 매개변수 : 주식 데이터 데이터프레임, 
#           시작 시간(2010-01-01), 
#           종료 시간(현재 시간), 
#           컬럼의 선택(Adj Close)

def buyandhold(
        _df, 
        _start = "2010-01-01", 
        _end = datetime.now(), 
        _col = 'Adj Close'
):
    # DataFrame을 복사 copy()
    result = _df.copy()
    # 인덱스가 0부터 시작하는 인덱스 // Date 인덱스 경우 
    # 데이터프레임에서 컬럼 중 Date라는 컬럼이 존재하는가?
    if "Date" in result.columns:
        # Date 컬럼을 인덱스로 변경 
        result.set_index('Date', inplace=True)
    # index를 시계열데이터로 변경
    result.index = pd.to_datetime(result.index)

    # _start, _end 값을 시계열 데이터로 변경 
    try : 
        start = datetime.strptime(_start, "%Y-%m-%d")
        # _end은 기본값은 시계열인 경우는 아무 행동도 하지 않는다. 
        # 문자열인 경우는 시계열로 데이터를 변경 
        if type(_end) == 'str':
            end = datetime.strptime(_end, '%Y-%m-%d')
        else:
            end = _end
    except:
        print(f"시작시간과 종료시간의 포멧은 YYYY-mm-dd 형식입니다")
        return ""
    
    # 시작시간과 종료시간을 기준으로 데이터를 필터링 -> 특정 컬럼만 필터링
    result = result.loc[start : end, [_col]]
    # 일별 수익율 컬럼을 생성 
    result['daily_rtn'] = (result[_col].pct_change() + 1).fillna(1)
    # 누적 수익율 컬럼을 생성
    result['acc_rtn'] = result['daily_rtn'].cumprod()
    print(f"""{start.strftime('%Y-%m-%d')}부터 
          {end.strftime('%Y-%m-%d')}까지 
          buyandhold의 수익율은 {result.iloc[-1, 2]}입니다""")
    # return데이터에 데이터프레임, 총 수익율 
    acc_rtn = result.iloc[-1, 2]
    return result, acc_rtn


In [ ]:
# AMZN 데이터를 로드 
AMZN = pd.read_csv("../../csv/AMZN.csv")
AMZN.head()

In [ ]:
bnh_amzn, rtn_amzn = buyandhold(AMZN)

In [ ]:
rtn_amzn

In [ ]:
bnh_amzn

In [ ]:
df = pd.read_csv("../../csv/GM.csv", index_col='Date')
df.head()

In [ ]:
buyandhold(df)

In [ ]:
buyandhold(df, _start ='2015-01-01')

In [59]:
buyandhold(df, _start = '2015/01/01')

시작시간과 종료시간의 포멧은 YYYY-mm-dd 형식입니다


''

In [60]:
# bnh 모듈을 로드 
import bnh

In [62]:
import importlib
importlib.reload(bnh)

<module 'bnh' from 'c:\\ubion_2025\\python\\250324\\bnh.py'>

In [61]:
bnh.buyandhold(df)

2010-01-01부터 
          2025-03-24까지 
          buyandhold의 수익율은 1.3621464972595223입니다


(            Adj Close  daily_rtn   acc_rtn
 Date                                      
 2010-11-18  27.027195   1.000000  1.000000
 2010-11-19  27.082527   1.002047  1.002047
 2010-11-22  26.940243   0.994746  0.996783
 2010-11-23  26.284130   0.975646  0.972507
 2010-11-24  26.465942   1.006917  0.979234
 ...               ...        ...       ...
 2019-06-18  36.700001   1.015495  1.357892
 2019-06-19  36.779999   1.002180  1.360852
 2019-06-20  36.959999   1.004894  1.367511
 2019-06-21  36.919998   0.998918  1.366031
 2019-06-24  36.814999   0.997156  1.362146
 
 [2162 rows x 3 columns],
 1.3621464972595223)